In [1]:
from cellpose import core, utils, io, models, metrics
from tifffile import imread, imwrite
from pathlib import Path
from skimage.transform import resize
import numpy as np
import napari
from tqdm.notebook import tqdm

In [2]:
image_path = Path("../DATA/series003_cCAR_tumor.tif")

image = imread(image_path)
image_actin_channel = image[:, 1, ...]
image = image[:, 0, ...]
image.shape

(162, 1412, 1412)

In [3]:
image_resized =  resize(image, (image.shape[0], image.shape[1] // 2, image.shape[2] // 2), anti_aliasing=True)
image_actin_resized = resize(image_actin_channel, (image_actin_channel.shape[0], image_actin_channel.shape[1] // 2, image_actin_channel.shape[2] // 2), anti_aliasing=True)

In [4]:
v = napari.Viewer()

c:\Users\Cyril\anaconda3\envs\napari-cellpose\lib\site-packages\napari_tools_menu\__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


In [5]:
v.add_image(image_resized, name="image", colormap="viridis")
v.add_image(image_actin_resized, name="actin", colormap="magma")

<Image layer 'actin' at 0x1e94259abb0>

In [6]:
cellpose_model = models.CellposeModel(gpu=False, model_type='cyto3')

C:\Users\Cyril\AppData\Roaming\Python\Python38\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
masks = np.zeros_like(image_resized, dtype=np.uint16)
for i in tqdm(range(image_resized.shape[0])):
    masks[i], flows, styles = cellpose_model.eval(
        image_resized[i], diameter=45, do_3D=False, channels=[0, 0], normalize=True, flow_threshold=0.6, cellprob_threshold=-1.0
    )
    if i > 0:
        break
masks = masks[0]

  0%|          | 0/162 [00:00<?, ?it/s]

In [8]:
v.add_labels(masks, name="masks")

<Labels layer 'masks' at 0x1e94408b1c0>

In [9]:
v.add_image(image[0], name="image", colormap="viridis")

<Image layer 'image [1]' at 0x1e9447fe610>

In [10]:
masked_actin = image_actin_resized[0]
masked_actin = np.where(masks == 0, masked_actin, 0)

In [11]:
v.add_image(masked_actin, name="masked_actin", colormap="magma")

<Image layer 'masked_actin' at 0x1e944839850>